In [1]:
import os

In [2]:
%pwd

'c:\\datascience\\Projects\\text_summarizer\\research'

In [3]:
import os
os.chdir(r"C:\datascience\Projects\text_summarizer")


In [4]:
%pwd

'C:\\datascience\\Projects\\text_summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directory,read_yaml_config,get_size
from textSummarizer.logger_config.config import logger
import urllib.request as request
import zipfile


Logger module loaded successfully!


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-04-07 10:35:23,559: INFO: Successfully loaded configuration from: config\config.yaml]
[2025-04-07 10:35:23,573: INFO: Successfully loaded configuration from: params.yaml]
[2025-04-07 10:35:23,575: INFO: Created directory at: artifacts]
[2025-04-07 10:35:23,577: INFO: Created directory at: artifacts/data_ingestion]
[2025-04-07 10:35:23,579: INFO: File already exists of size: ~ 7718 KB]


In [10]:
from textSummarizer.utils.common import read_yaml_config
from pathlib import Path

cfg = read_yaml_config(Path("config/config.yaml"))
print(cfg)


[2025-04-07 10:35:28,324: INFO: Successfully loaded configuration from: config\config.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'data_path': 'artifacts/data_transformation/sams